<a href="https://colab.research.google.com/github/jgaffiot/rag-europython-2025/blob/main/template_routing_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic RAG routing using Hybrid Search

## Initial setup

### Copy this notebook

- Click on `File` from the Navbar.
- Select `Save a copy in Drive`.
- Open the new notebook in a new tab and keep it ready during the workshop session.


### Installation

In [1]:
!pip install llama-index
!pip install llama-index-llms-google-genai
!pip install opik llama-index-callbacks-opik
!pip install llama-index-vector-stores-qdrant
!pip install llama-index-embeddings-fastembed
!pip install pycountry googlesearch-python
!pip install gradio

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found exi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.7/657.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.

### Setup OPIK Tracing, LLM and Embedding model credentials

> OPIK API KEY

- Get your OPIK API key: [Comet- OPIK for tracing and monitoring](https://www.comet.com/signup?utm_source=workshop&utm_medium=partner&utm_campaign=tarun )
- Login via Google/GitHub and copy the API key.

> Google API Key

- Get your Google API key: [Gemini 2.5 Flash](https://aistudio.google.com/)



In [1]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")
os.environ['OPIK_API_KEY'] = userdata.get("OPIK_API_KEY")
os.environ["OPIK_WORKSPACE"] = "jonathan-gaffiot"
os.environ["OPIK_PROJECT_NAME"] = "EuroPython2025"

In [2]:
import nest_asyncio
nest_asyncio.apply()

### Download data

In [3]:
url = "https://ncert.nic.in/textbook/pdf/iess302.pdf"
book_path = "socialism.pdf"

In [4]:
import httpx

with httpx.Client() as client:
    response = client.get(url)
    response.raise_for_status()  # Raise an error for bad status codes
    with open(book_path, 'wb') as file:
        file.write(response.content)

## RAG development

### Setup models

In [6]:
from llama_index.core import Settings
from opik.integrations.llama_index import LlamaIndexCallbackHandler
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.fastembed import FastEmbedEmbedding

In [8]:
llm_name = "gemini-2.5-flash"
embedding_model_name = "jinaai/jina-embeddings-v2-base-en"

In [9]:
opik_callback_handler = LlamaIndexCallbackHandler()

In [10]:
llm = GoogleGenAI(model=llm_name)

In [11]:
embedding_model = FastEmbedEmbedding(model_name=embedding_model_name)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

onnx/model.onnx:   0%|          | 0.00/547M [00:00<?, ?B/s]

In [12]:
llm.complete("what'up ?")

CompletionResponse(text="Hey there! Not much, just here and ready to help.\n\nWhat's up with you? How can I assist you today?", additional_kwargs={}, raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'inline_data': None, 'file_data': None, 'thought_signature': None, 'code_execution_result': None, 'executable_code': None, 'function_call': None, 'function_response': None, 'text': "Hey there! Not much, just here and ready to help.\n\nWhat's up with you? How can I assist you today?"}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'url_context_metadata': None, 'avg_logprobs': None, 'grounding_metadata': None, 'index': 0, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 28, 'candidates_tokens_details': None, 'prompt_token_count': 5, 'prompt_tokens_details': [{'modality': <Me

In [19]:
from llama_index.core.callbacks import CallbackManager

Settings.llm = llm
Settings.embed_model = embedding_model
Settings.callback_manager = CallbackManager([opik_callback_handler])

### Google Search

In [18]:
from googlesearch import search as google_search
from llama_index.core.tools import FunctionTool

In [28]:
response = google_search("spam", num_results=10, advanced=True)
print("\n".join(result.description for result in response))

 Enjoy the best canned meat meals using easy recipes and a variety of delicious, high-quality SPAM® meat. See what SPAM® Brand can do! 
 Spam (stylized in all-caps) is a brand of lunch meat (processed canned pork and ham) made by Hormel Foods Corporation, an American multinational food ... 
 It's made with six simple ingredients and has 7 g of protein per serving. Includes one 12 oz aluminum can of SPAM Classic. 
 The CAN-SPAM Act, a law that sets the rules for commercial email, establishes requirements for commercial messages, gives recipients the right to have you stop ... 
 Spam is a brand of lunch meat made by Hormel Foods Corporation, an American multinational food processing company. It was introduced in the United States in 1937 and gained popularity worldwide after its use during World War II.   Wikipedia 
 The word "Spam" as applied to Email means "Unsolicited Bulk Email". Unsolicited means that the Recipient has not granted verifiable permission for the message ... 

 In the 

In [30]:
def search(query: str, *, max_result: int = 10) -> str:
  """
  context: use this tool to search for general information over the Internet,
  using Google Search.
  """
  context = ""
  response = google_search(query, num_results=max_result, advanced=True)
  return "\n".join(result.description for result in response)


In [31]:
search_tool = FunctionTool.from_defaults(fn=search)

In [32]:
search_tool.call("spam")

ToolOutput(blocks=[TextBlock(block_type='text', text=' Enjoy the best canned meat meals using easy recipes and a variety of delicious, high-quality SPAM® meat. See what SPAM® Brand can do! \n Spam (stylized in all-caps) is a brand of lunch meat (processed canned pork and ham) made by Hormel Foods Corporation, an American multinational food\xa0... \n It\'s made with six simple ingredients and has 7 g of protein per serving. Includes one 12 oz aluminum can of SPAM Classic. \n Spam is a brand of lunch meat made by Hormel Foods Corporation, an American multinational food processing company. It was introduced in the United States in 1937 and gained popularity worldwide after its use during World War II.   Wikipedia \n The CAN-SPAM Act, a law that sets the rules for commercial email, establishes requirements for commercial messages, gives recipients the right to have you stop\xa0... \n\n The word "Spam" as applied to Email means "Unsolicited Bulk Email". Unsolicited means that the Recipient 

### Retrieval bricks

In [34]:
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex,StorageContext
from qdrant_client import QdrantClient

In [48]:
collection_name = "history_v1"

In [38]:
documents = SimpleDirectoryReader(input_files=[book_path], recursive=True).load_data()
len(documents)

24

In [39]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embedding_model
)
nodes = splitter.get_nodes_from_documents(documents)

In [50]:
len(nodes)

63

In [51]:
nodes[0]

TextNode(id_='7b779cd5-9016-4e51-b60a-6743c0d2bc98', embedding=None, metadata={'page_label': '1', 'file_name': 'socialism.pdf', 'file_path': 'socialism.pdf', 'file_type': 'application/pdf', 'file_size': 5508757, 'creation_date': '2025-07-15', 'last_modified_date': '2025-07-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='49469fba-c47d-4018-8b7a-e8f3c8893358', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'socialism.pdf', 'file_path': 'socialism.pdf', 'file_type': 'application/pdf', 'file_size': 5508757, 'creation_date': '2025-07-15', 'last_modified_date': '2025-07-15'}, hash='1468a44615f11c0e6922924762ddd4bdb7ff9f6d632b8da455f04649ef222acb'), <NodeRelationship.NEXT: '3

In [52]:
documents[0]

Document(id_='49469fba-c47d-4018-8b7a-e8f3c8893358', embedding=None, metadata={'page_label': '1', 'file_name': 'socialism.pdf', 'file_path': 'socialism.pdf', 'file_type': 'application/pdf', 'file_size': 5508757, 'creation_date': '2025-07-15', 'last_modified_date': '2025-07-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Socialism in Europe and the Russian Revolution\n25\nSocialism in Europe andSocialism in Europe andSocialism in Europe andSocialism in Europe andSocialism in Europe and\nthe Russian Revolutionthe Russian Revolutionthe Russian Revolutionthe Russian Revolutionthe Russian Revolution\n1  The Age of Social Change\nIn the pr

In [46]:
import tempfile

tmp_dir=tempfile.TemporaryDirectory()
print(tmp_dir)
qdran_client = QdrantClient(path=tmp_dir.name)

<TemporaryDirectory '/tmp/tmppxgybakt'>


In [49]:
vector_store = QdrantVectorStore(
    client=qdran_client,
    collection_name=collection_name,
    enable_hybrid=True,
    fastembed_sparse_model = "Qdrant/bm25",
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

arabic.txt: 0.00B [00:00, ?B/s]

finnish.txt: 0.00B [00:00, ?B/s]

danish.txt:   0%|          | 0.00/424 [00:00<?, ?B/s]

dutch.txt:   0%|          | 0.00/453 [00:00<?, ?B/s]

...

hungarian.txt: 0.00B [00:00, ?B/s]

romanian.txt: 0.00B [00:00, ?B/s]

spanish.txt: 0.00B [00:00, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

In [53]:
index = VectorStoreIndex(nodes, storage_context=storage_context)

/usr/local/lib/python3.11/dist-packages/llama_index/vector_stores/qdrant/base.py:709: UserWarning: Payload indexes have no effect in the local Qdrant. Please use server Qdrant if you need payload indexes.
  self._client.create_payload_index(
OPIK: Started logging traces to the "EuroPython2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01980d7a-452e-76ca-9542-759e3a1059e7&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


In [57]:
list(Path(tmp_dir.name).iterdir())

[PosixPath('/tmp/tmppxgybakt/collection'),
 PosixPath('/tmp/tmppxgybakt/.lock'),
 PosixPath('/tmp/tmppxgybakt/meta.json')]

In [62]:
query_engine = index.as_query_engine()

In [70]:
query_engine.get_prompts().keys()

dict_keys(['response_synthesizer:text_qa_template', 'response_synthesizer:refine_template'])

In [63]:
user_query = "what is the russian revolution ?"

In [64]:
response = query_engine.query(user_query)

In [69]:
response.response

"The Russian Revolution refers to a series of significant events, including the 1905 Revolution and the 1917 revolutions.\n\nThe 1905 Revolution began with an incident known as Bloody Sunday, which led to widespread strikes across the country. Universities closed due to student walkouts, and various professional groups formed the Union of Unions, demanding a constituent assembly. This period resulted in the Tsar allowing the creation of an elected consultative Parliament, or Duma.\n\nIn 1917, a socialist seizure of power occurred, orchestrated by the Bolshevik Party and the Petrograd Soviet. An uprising began on October 24th, leading to the Military Revolutionary Committee seizing government offices and arresting ministers. The ship Aurora shelled the Winter Palace, and by nightfall, the city was under the committee's control. The Bolshevik action was approved by the All Russian Congress of Soviets, and heavy fighting ensued in other cities, with the Bolsheviks gaining control of the M

In [68]:
response.get_formatted_sources()

'> Source (Doc id: 7f1833d3-f099-4a0c-a2f6-7e29fac513ca): Socialism in Europe and the Russian Revolution\n33\nActivity\nparliament. Liberals in Russia campaig...\n\n> Source (Doc id: c009c949-3ccd-4be4-aee7-81697659e1ea): Bolshevik supporters in the army,\nsoviets and  factories were brought together.\nOn 16 October 191...'

In [72]:
response.source_nodes[0].node.get_content()

'Socialism in Europe and the Russian Revolution\n33\nActivity\nparliament. Liberals in Russia campaigned to end this state of affairs.\nTogether with the Social Democrats and Socialist Revolutionaries,\nthey worked with peasants and workers during the revolution of\n1905 to demand a constitution. They were supported in the empire\nby nationalists (in Poland for instance) and in Muslim-dominated\nareas by \njadidists who wanted modernised Islam to lead their societies.\nThe year 1904 was a particularly bad one for Russian workers. Prices\nof essential goods rose so quickly that real wages declined by 20 per\ncent. The membership of workers’ associations rose dramatically.\nWhen four members of the Assembly of Russian Workers, which\nhad been formed in 1904, were dismissed at the Putilov Iron Works,\nthere was a call for industrial action. Over the next few days over\n110,000 workers in St Petersburg went on strike demanding a\nreduction in the working day to eight hours, an increase in 

In [73]:
inference_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
 #    storage_context=storage_context,
)

In [74]:
inference_query_engine = inference_index.as_query_engine(
    vector_store_query_mode="hybrid",
    sparse_top_k=3,
    similarity_top_k=3,
    hybrid_top_k=3,
)

In [75]:
inference_query_engine.query(user_query).response

'The Russian Revolution refers to the fall of the monarchy in February 1917 and the subsequent events of October 1917, when socialists took control of the government in Russia.'

### Knwoledge Base

In [77]:
def knowledge_base(query: str) -> str:
  """
  context: use this tool to search in a knowledge base about the Russian Revolution.
  """
  inference_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    # storage_context=storage_context,
    )
  inference_query_engine = inference_index.as_query_engine(
    vector_store_query_mode="hybrid",
    sparse_top_k=3,
    similarity_top_k=3,
    hybrid_top_k=3,
  )
  response = inference_query_engine.query(query)
  return response.response

In [78]:
kb_tool = FunctionTool.from_defaults(fn=knowledge_base)

In [79]:
kb_tool.call(user_query)

ToolOutput(blocks=[TextBlock(block_type='text', text='The Russian Revolution refers to the fall of the monarchy in February 1917 and the subsequent events of October 1917, when socialists took control of the government in Russia.')], tool_name='knowledge_base', raw_input={'args': ('what is the russian revolution ?',), 'kwargs': {}}, raw_output='The Russian Revolution refers to the fall of the monarchy in February 1917 and the subsequent events of October 1917, when socialists took control of the government in Russia.', is_error=False)

### Router agent

In [92]:
context = query_engine.query("what is the russian revolution ?")

In [93]:
len(context.source_nodes)

2

In [95]:
print(context.source_nodes[0].text)

Socialism in Europe and the Russian Revolution
33
Activity
parliament. Liberals in Russia campaigned to end this state of affairs.
Together with the Social Democrats and Socialist Revolutionaries,
they worked with peasants and workers during the revolution of
1905 to demand a constitution. They were supported in the empire
by nationalists (in Poland for instance) and in Muslim-dominated
areas by 
jadidists who wanted modernised Islam to lead their societies.
The year 1904 was a particularly bad one for Russian workers. Prices
of essential goods rose so quickly that real wages declined by 20 per
cent. The membership of workers’ associations rose dramatically.
When four members of the Assembly of Russian Workers, which
had been formed in 1904, were dismissed at the Putilov Iron Works,
there was a call for industrial action. Over the next few days over
110,000 workers in St Petersburg went on strike demanding a
reduction in the working day to eight hours, an increase in wages
and improvem

In [96]:
some_context = context.source_nodes[0].text

In [99]:
SYSTEM_PROMPT_TEMPLATE = """
You are an expert routing Agent, you first check the user query if its relevant
to the knowledge base or not.
Knowledge base is mainly about Socialism in Europe and the Russian Revolution
and refer to more CONTEXT. If user query is not relevant to the context then
use search_tool.

Now if the user query overlaps and is not retrieved from the Knowledge_base_tool
that means the information is not present in the knowledge_base.
Then you need to trigger the search tool to get the results from web via Google
Search_tool.

CONTEXT:
{some_context}
"""

In [100]:
system_prompt = SYSTEM_PROMPT_TEMPLATE.format(some_context=some_context)

In [109]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.workflow import Context

agent = FunctionAgent(
    tools=[search_tool, kb_tool],
    llm=llm,
    system_prompt=system_prompt,
    verbose=True,
)
context = Context(agent)

In [111]:
response = await agent.run("what is the russian revolution ?", ctx=context)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_ag

In [122]:
response.response.blocks[0].text

'The Russian Revolution refers to the fall of the monarchy in February 1917 and the subsequent events of October 1917, when socialists took control of the government in Russia.\n\nA significant event leading up to the 1905 Revolution, a precursor to the main Russian Revolution, was "Bloody Sunday." This incident occurred when a procession of workers, led by Father Gapon, was attacked by police and Cossacks near the Winter Palace, resulting in over 100 deaths and 300 injuries. This event sparked a series of strikes and protests across the country, leading to the Tsar allowing the creation of an elected consultative Parliament or Duma.'

In [115]:
[tc.tool_name for tc in response.tool_calls]

['knowledge_base', 'knowledge_base']

In [123]:
response2 = await agent.run("who is Messi ?", ctx=context)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [125]:
response2.response.blocks[0].text

"Lionel Messi is an Argentine professional footballer who is widely regarded as one of the greatest players of all time. He was born in Rosario, Argentina, in 1987. He currently plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.\n\nMessi is the Argentine national team's all-time leading goalscorer and most-capped player. He has received a record-setting eight Ballon d'Or awards, which is given to the world's best soccer player. He also founded the Leo Messi Foundation in 2007."

In [124]:
[tc.tool_name for tc in response2.tool_calls]

['search']

## UI

In [126]:
import gradio as gr
import asyncio

In [127]:
async def chat_response_async(message, history, agent):
    if not message.strip():
        return history, ""

    try:
        ctx = Context(agent)
        result = await agent.run(message, ctx=ctx)

        response = result.response.blocks[0].text
        tool_used = result.tool_calls[0].tool_name if result.tool_calls else "direct"

        history.append([message, f"{response}\n\n*Tool: {tool_used}*"])
        return history, ""

    except Exception as e:
        history.append([message, f"Error: {str(e)}"])
        return history, ""

def chat_response(message, history, agent):
    return asyncio.run(chat_response_async(message, history, agent))

In [128]:
def launch_chat_app(agent):
    with gr.Blocks() as demo:
        gr.Markdown("# Socialist History Knowledge Agent")

        chatbot = gr.Chatbot()
        msg = gr.Textbox(placeholder="Ask your question...", label="Message")

        msg.submit(
            lambda m, h: chat_response(m, h, agent),
            [msg, chatbot],
            [chatbot, msg]
        )

    demo.launch(share=True)
    return demo

launch_chat_app(agent)

/tmp/ipython-input-128-3532884132.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3be05b3ad98c67572e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Gradio Blocks instance: 1 backend functions
-------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x7aeac29477d0>
 |-<gradio.components.chatbot.Chatbot object at 0x7aeac2e0f490>
 outputs:
 |-<gradio.components.chatbot.Chatbot object at 0x7aeac2e0f490>
 |-<gradio.components.textbox.Textbox object at 0x7aeac29477d0>